# What this file does
1. extract the 100 random genes
2. point to the genome region so that they can be stripped using samtools as in `exon_data_pull.ipynb`
3. create k-mers for those random genes `'/cellar/users/ramarty/Data/kir/kmers/reference_genes/random_genes.100.txt'`

In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/kir/KIR_development/bin/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/kir/KIR_development/bin/samples.ipynb")

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated since IPython 4.0. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)
/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


### Make set of random genes

In [67]:
input_file = '/cellar/users/mdow/Data/hg38/hg38_ensembl.fa'

In [69]:
names, sequences, chromosomes, starts, ends = [], [], [], [], []
fasta_sequences = SeqIO.parse(open(input_file),'fasta')
for fasta in fasta_sequences:
    try:
        name, sequence = fasta.id, fasta.seq.tostring()
        if int(fasta.description.split('chromosome:')[1].split(':')[1]):
            fasta.description.split('chromosome:')[1].split(':')[3]
            names.append(name)
            sequences.append(sequence)
            chromosomes.append(fasta.description.split('chromosome:')[1].split(':')[1])
            starts.append(fasta.description.split('chromosome:')[1].split(':')[2])
            ends.append(fasta.description.split('chromosome:')[1].split(':')[3])
    except:
        None

In [70]:
df = pd.DataFrame({'Gene': names,
                  'Sequences': sequences,
                  'Chromosome': chromosomes,
                  'Start': starts,
                  'End': ends})

In [71]:
random_genes = df.sample(100)

In [72]:
random_genes.head()

,Chromosome,End,Gene,Sequences,Start
19110,19,19106934,ENST00000594070,AGTACACACAGGACCGTGGCTGCGGAATGCAGTTGAGGGCATCCCT...,19095943
58862,11,47642452,ENST00000539759,CCAGTCAGGTGCTCCTGGGCTCCGGTCTCACCATCCTGTCCCAGCC...,47617315
41407,2,75561323,ENST00000486489,CTGAATACAGGCTCAAAGCAGGCGACGGGACAGGAATGAGAGGATG...,75558483
14229,1,154968874,ENST00000444179,GGCAGGCAGGCGCTGGGTCTGAGTACCCCTCTTTCCCCAGGGAGGC...,154966425
55814,4,174304280,ENST00000503053,TGGCTGTAGCGACTGGTTGCGGGAGGCTGGACTCTGGTGACTGTGT...,174284018


In [73]:
for row in random_genes.iterrows():
    print row[1]['Chromosome']
    break

19


In [74]:
with open('/cellar/users/ramarty/Data/kir/ref/random_genes.fa', 'w') as outfile:
    for row in random_genes.iterrows():
        outfile.write('>{0}, chr{1}:{2}-{3}\n'.format(row[1]['Gene'], row[1]['Chromosome'], 
                                                      row[1]['Start'], row[1]['End']))
        outfile.write('{0}\n'.format(row[1]['Sequences']))

Merge with all_alleles and build <br>
in: /data/cellardata/users/ramarty/kir/ref <br>
cat all_alleles random_genes.fa > all_alleles_and_random <br>
/cellar/users/ramarty/programs/bowtie2-2.2.9/bowtie2-build all_alleles_and_random all_alleles_and_random

### Get the regions of the random genes

In [2]:
# Get the regions of the 
input_file = '/cellar/users/ramarty/Data/kir/ref/random_genes.fa'
names, sequences = [], []
fasta_sequences = SeqIO.parse(open(input_file),'fasta')
for fasta in fasta_sequences:
    name, sequence = fasta.description, fasta.seq.tostring()
    names.append(name)
    sequences.append(sequence)

df = pd.DataFrame({'Gene': names,
                  'Sequences': sequences})

fasta.description

regions = [x.split(', ')[1] for x in df.Gene]

regions_str = ' '.join(regions)

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/Bio/Seq.py:354: BiopythonDeprecationWarning: This method is obsolete; please use str(my_seq) instead of my_seq.tostring().
  BiopythonDeprecationWarning)


### Make k-mers in line with those genes <br>
Test of a single one to make sure my alignment isn't crap <br>
Must do all other ones later

In [2]:
input_file = '/cellar/users/ramarty/Data/kir/ref/random_genes.fa'
names, sequences = [], []
fasta_sequences = SeqIO.parse(open(input_file),'fasta')
for fasta in fasta_sequences:
    name, sequence = fasta.description, fasta.seq.tostring()
    names.append(name)
    sequences.append(sequence)

df = pd.DataFrame({'Gene': names,
                  'Sequences': sequences})


/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/Bio/Seq.py:354: BiopythonDeprecationWarning: This method is obsolete; please use str(my_seq) instead of my_seq.tostring().
  BiopythonDeprecationWarning)


In [8]:
kmers = []
for sequence in df.Sequences:
    entire_set = []
    for i in range(len(sequence)-30):
        entire_set.append(sequence[i:i+30])
    if len(entire_set) > 100:
        kmers.extend(pd.Series(entire_set).sample(100))
    else:
        kmers.extend(entire_set)

In [10]:
len(kmers)

9936

In [11]:
with open('/cellar/users/ramarty/Data/kir/kmers/reference_genes/random_genes.100.txt', 'w') as outfile:
    for kmer in kmers:
        outfile.write('{0}\n'.format(kmer))

Figure out which goes with which?